## Contextual Compression Retriever란?
- 검색된 문서에서 쿼리와 관련된 정보만을 압축하여 추출하는 Retriever
- 기본 Retriever가 검색한 문서를 Document Compressor가 압축하여 품질을 향상
- 불필요한 정보를 제거하여 LLM에 전달되는 정보의 노이즈 개선

**기존 RAG 문제점**
1. **관련 없는 정보**: 청크에 쿼리와 무관한 정보가 포함되어 LLM을 혼란시킴
2. **토큰 낭비**: 긴 문서가 프롬프트 공간을 차지하여 비용 증가
3. **정확도 저하**: 불필요한 정보로 인한 응답 품질 저하

**핵심 개념**
- **Base Retriever**: 기본 문서 검색을 담당하는 Retriever (VectorStore 등)
- **Document Compressor**: 검색된 문서를 압축/필터링하는 구성요소


실습할 문서 만들기

In [ ]:
from langchain_core.documents import Document

# 군용 무기 관련 상세 정보가 포함된 샘플 문서 생성
documents = [
    Document(
        page_content="K2 흑표는 대한민국이 개발한 3.5세대 주력전차로, 120mm 주포와 복합장갑을 갖추고 있습니다. 현재 대한민국 육군의 주력전차로 운용되고 있으며, 폴란드 등 해외에도 수출되고 있습니다. 전차의 무게는 55톤이며, 최고속도는 70km/h입니다. 또한 능동방어시스템과 전투지휘체계가 탑재되어 있어 현대전에 최적화되어 있습니다.",
        metadata={"source": "국방백서", "category": "기갑차량", "년도": "2021"}
    ),
    Document(
        page_content="K9 자주포는 155mm 구경의 자주포로, 최대 사거리 40km를 자랑하는 한국형 자주포입니다. 분당 6발의 연사속도를 가지고 있으며, 완전 자동화된 탄약공급시스템을 갖추고 있습니다. 또한 디지털 사격통제시스템을 통해 정밀한 타격이 가능합니다. 현재 전 세계 10여 개국에 수출되어 운용 중입니다.",
        metadata={"source": "방산업체", "category": "포병장비", "년도": "2020"}
    ),
    Document(
        page_content="KAI KF-21 보라매는 한국형 전투기 사업으로 개발된 4.5세대 다목적 전투기입니다. 쌍발엔진을 장착하고 있으며, 최고속도는 마하 1.8입니다. AESA 레이더와 첨단 전자장비를 탑재하여 공대공, 공대지 임무를 모두 수행할 수 있습니다. 현재 시제기 개발이 완료되어 양산 준비 중입니다.",
        metadata={"source": "항공우주", "category": "항공기", "년도": "2022"}
    ),
    Document(
        page_content="천궁 방공미사일은 중거리 지대공 미사일로 탄도미사일 요격 능력을 보유하고 있습니다. 사거리는 40km이며, 다중표적 동시교전이 가능합니다. 패트리어트 미사일보다 우수한 성능을 가지고 있다고 평가받고 있습니다. 현재 UAE에 수출 계약이 체결되어 있으며, 추가 수출 협상이 진행 중입니다.",
        metadata={"source": "방위산업", "category": "미사일", "년도": "2021"}
    ),
    Document(
        page_content="독도함은 대형수송함으로 상륙작전 지원과 헬기 운용이 가능한 다목적 함정입니다. 전장 199m, 만재배수량 18,800톤의 대형함정으로 해병대 상륙작전을 지원합니다. 함재기로는 상륙헬기와 공격헬기를 운용할 수 있으며, 최대 720명의 해병대원을 수송할 수 있습니다. 현재 독도함과 마라도함 2척이 운용 중입니다.",
        metadata={"source": "해군본부", "category": "함정", "년도": "2020"}
    ),
    Document(
        page_content="수리온 헬기는 한국형 기동헬기로 다양한 임무에 활용되는 다목적 헬리콥터입니다. 최대 수송능력은 9명이며, 최고속도는 324km/h입니다. 의료후송, 수색구조, 특수임무 등에 활용되고 있습니다. 또한 민수용으로도 개발되어 소방청, 경찰청 등에서 운용하고 있습니다. 해외 수출도 추진되고 있어 태국 등에 수출 계약을 체결했습니다.",
        metadata={"source": "육군본부", "category": "헬리콥터", "년도": "2019"}
    ),
    Document(
        page_content="현무 미사일은 지대지 전술미사일로 정밀타격 능력을 갖춘 한국형 미사일입니다. 현무-2는 사거리 300km, 현무-3는 사거리 1,500km를 가지고 있습니다. GPS와 INS를 이용한 정밀유도시스템을 갖추고 있어 CEP 10m 이내의 정확도를 자랑합니다. 북한의 핵시설과 미사일 기지를 타격할 수 있는 핵심 전력으로 평가받고 있습니다.",
        metadata={"source": "국방과학연구소", "category": "미사일", "년도": "2021"}
    ),
    Document(
        page_content="세종대왕급 구축함은 이지스 시스템을 탑재한 대한민국 해군의 최신 구축함입니다. SPY-1D 레이더와 SM-2 미사일을 장착하여 광역방공능력을 보유하고 있습니다. 전장 166m, 만재배수량 8,500톤의 대형 구축함으로 총 3척이 건조되어 운용 중입니다. 또한 한국형 차세대 구축함(KDDX) 사업을 통해 후속함 건조가 예정되어 있습니다.",
        metadata={"source": "해군본부", "category": "함정", "년도": "2020"}
    )
]

print(f"준비된 군용 무기 문서 수: {len(documents)}")
print(f"첫 번째 문서 길이: {len(documents[0].page_content)}자")

임베딩 객체 생성

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

# 한국어 임베딩 모델 초기화
embeddings = HuggingFaceEmbeddings(
    model_name="nlpai-lab/KURE-v1",  # 한국어 임베딩 모델
    model_kwargs={'device': 'cpu'},  # CPU 사용
    encode_kwargs={'normalize_embeddings': True}  # 벡터 정규화
)

print(f"임베딩 모델: {embeddings.model_name}")


Vector Store 생성

In [ ]:
from langchain_community.vectorstores import FAISS

# FAISS 벡터 스토어 생성
vectorstore = FAISS.from_documents(
    documents=documents, 
    embedding=embeddings
)

print("VectorStore가 성공적으로 생성되었습니다.")


기본 VectorStore Retriever 테스트

In [ ]:
# 기본 VectorStore Retriever 생성
base_retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# 기본 검색 테스트
question = "미사일의 사거리는 얼마나 되나요?"
docs = base_retriever.invoke(question)

print(f"=== 기본 검색 결과 (총 {len(docs)}개) ===")

for i, doc in enumerate(docs):
    print(f"문서 {i+1}:")
    print(f"내용: {doc.page_content}")
    print(f"문서 길이: {len(doc.page_content)}자")
    print(f"카테고리: {doc.metadata['category']}")

### LLMChainExtractor를 사용한 Contextual Compression

**LLMChainExtractor**
- LLM을 이용하여 문서나 텍스트로부터 정보(예: 키워드, 요약, 질문 등)를 추출하는 유틸
- LLM + Prompt + OutputParser를 묶어서, 텍스트에서 원하는 정보를 추출 할 수도 있음.

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

# LLM 초기화
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="midm-2.0-base-instruct-q5_k_m",
    temperature=0, # 창의성 (0.0 ~ 1.0)
)

# LLMChainExtractor 생성
# 문서에서 쿼리와 관련된 정보만을 추출
compressor = LLMChainExtractor.from_llm(llm)

# ContextualCompressionRetriever 생성
compression_retriever = ContextualCompressionRetriever(

    base_compressor=compressor,
    base_retriever=base_retriever
)

In [ ]:
# 압축된 검색 결과
compressed_docs = compression_retriever.invoke(question)

print(f"LLMChainExtractor 압축 결과 (총 {len(compressed_docs)}개)")
for i, doc in enumerate(compressed_docs):
    print(f"압축된 문서 {i+1}:")
    print(f"내용: {doc.page_content}")
    print(f"압축된 문서 길이: {len(doc.page_content)}자")
    print(f"카테고리: {doc.metadata['category']}")

### EmbeddingsFilter를 사용한 압축
EmbeddingsFilter는 LLM 호출 없이 임베딩 유사도만으로 문서를 필터링하여 비용을 절약합니다.

In [ ]:
from langchain.retrievers.document_compressors import EmbeddingsFilter

# EmbeddingsFilter 생성
embeddings_filter = EmbeddingsFilter(
    embeddings=embeddings,
    similarity_threshold=0.6,  # 유사도 임계값
    k=3  # 최대 반환 문서 수
)

# EmbeddingsFilter를 사용한 ContextualCompressionRetriever
embeddings_compression_retriever = ContextualCompressionRetriever(
    base_compressor=embeddings_filter,
    base_retriever=base_retriever
)

# 임베딩 필터링 결과
filtered_docs = embeddings_compression_retriever.invoke(question)

print(f"EmbeddingsFilter 결과 (총 {len(filtered_docs)}개) ===")
for i, doc in enumerate(filtered_docs):
    print(f"\n필터링된 문서 {i+1}:")
    print(f"내용: {doc.page_content}")
    print(f"카테고리: {doc.metadata['category']}")
